In [2]:
!pip install dash

# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

     |████████████████████████████████| 9.8 MB 11.1 MB/s eta 0:00:01          | 552 kB 11.1 MB/s eta 0:00:01███                   | 4.0 MB 11.1 MB/s eta 0:00:01█████████████████▏            | 5.9 MB 11.1 MB/s eta 0:00:010:01
     |████████████████████████████████| 95 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 357 kB 21.9 MB/s eta 0:00:01


In [3]:
spacex_df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv', index_col=False)
spacex_df.drop(columns='Unnamed: 0', inplace=True)
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [5]:
# Create a dash application
from cProfile import label
from numpy import float64


app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                 {'label': 'All Sites',
                                                     'value': 'ALL'},
                                                 {'label': 'CCAFS LC-40',
                                                     'value': 'CCAFS LC-40'},
                                                 {'label': 'VAFB SLC-4E',
                                                     'value': 'VAFB SLC-4E'},
                                                 {'label': 'KSC LC-39A',
                                                     'value': 'KSC LC-39A'},
                                                 {'label': 'CCAFS SLC-40',
                                                  'value': 'CCAFS SLC-40'},
                                             ],
                                             value='ALL',
                                             placeholder="Select a Launch Site here",
                                             searchable=True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),

                                html.Br(),

                                html.P("Payload range (Kg):"),

                                # TASK 3: Add a slider to select payload range
                                # dcc.RangeSlider(id='payload-slider',...)

                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       2500: '2500',
                                                       5000: '5000',
                                                       7500: '7500',
                                                       10000: '10000'
                                                       },
                                                value=[min_payload, max_payload]),


                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(
                                    dcc.Graph(
                                        id='success-payload-scatter-chart')
),
])

# # Function decorator to specify function input and output


@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        spacex_df_class = spacex_df.groupby(
            ['class'])['Launch Site'].count().reset_index()
        fig = px.pie(spacex_df_class, values='Launch Site',
                     names=spacex_df_class['class'],
                     title='Total Success Launches By Site'
                     )
        return fig
    else:
        # return the outcomes piechart for a selected site
        spacex_df_class_ent = spacex_df[spacex_df['Launch Site'] == entered_site].groupby(
            ['class'])['Launch Site'].count().reset_index()
        fig = px.pie(spacex_df_class_ent, values='Launch Site',
                     names=spacex_df_class_ent['class'],
                     title='Total Success Launches By Site'
                     )
        return fig


@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property='value')])
def get_scatter_chart(entered_site, payload):
    if entered_site == 'ALL':
        spacex_df_sc = spacex_df[['Payload Mass (kg)', 'class', 'Booster Version Category', 'Launch Site']][(
            spacex_df['Payload Mass (kg)'] <= payload[1]) & (spacex_df['Payload Mass (kg)'] >= payload[0])]
        fig_s = px.scatter(spacex_df_sc,
                           x='Payload Mass (kg)',
                           y='class',
                           color='Booster Version Category',
                           title='Correlation Between Payload and Success for All Sites'
                           )
        fig_s.update_yaxes(tick0=0, dtick=1)
        fig_s.update_xaxes(tick0=0, dtick=1000)
        return fig_s
    else:
        spacex_df_sc = spacex_df[['Payload Mass (kg)', 'class', 'Booster Version Category', 'Launch Site']][(
            spacex_df['Payload Mass (kg)'] <= payload[1]) & (spacex_df['Payload Mass (kg)'] >= payload[0]) & (spacex_df['Launch Site'] == entered_site)]
        fig_s = px.scatter(spacex_df_sc,
                           x='Payload Mass (kg)',
                           y='class',
                           color='Booster Version Category',
                           title=f'Correlation Between Payload and Success for {entered_site}'
                           )
        fig_s.update_yaxes(tick0=0, dtick=1)
        fig_s.update_xaxes(tick0=0, dtick=1000)
        return fig_s


In [6]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
